In [1]:
import pandas as pd
import numpy as np
import os
from pathlib import Path
from scipy import io

In [2]:
pwd

'/home/kimhj9502/Project/project_tDCS/analysis'

In [3]:
# Set directories
#DATA_DIR = Path('/home/kimhj9502/Project/project_tDCS/fMRI_task/data')
#REG_DIR = Path('/home/kimhj9502/Project/project_tDCS/analysis/fmri/data/regs') 
#OUT_DIR = Path('/home/kimhj9502/Project/project_tDCS/analysis/fmri/data/events')

In [4]:
# Set directories
DATA_DIR = Path('/home/kimhj9502/Project/project_tDCS/fMRI_task/data')
REG_DIR = Path('/data2/project_BRL/project_tDCS/behav/modeling/regs') 
OUT_DIR = Path('/data2/project_BRL/project_tDCS/events')

In [5]:
#subs = np.unique([np.int(s.split('_')[0].split('sub-')[-1]) for s in os.listdir(DATA_DIR) if '.tsv' in s])

In [17]:
anode_first = [28, 18, 33, 60, 63, 64, 76, 91, 86, 59, 105, 107, 109, 112, 121, 127] # 14, 61, 103, 116, 126
sham_first = [30, 54, 43, 11, 73, 79, 101, 108, 110, 104, 125] # 4, 15, 27, 69, 94, 102, 113
subs = anode_first + sham_first

In [18]:
len(subs)

27

In [15]:
len(anode_first)

16

In [16]:
len(sham_first)

11

In [7]:
#subs_brl = []
#for sub in subs:
#    tmp = 'sub-BRL'+'{:04d}'.format(sub)
#    subs_brl.append(tmp)

In [19]:
# Load model regressors
Qgo_anode = pd.read_csv(REG_DIR / 'Qgo_anode.csv', index_col = 0); Qgo_anode = Qgo_anode.T; Qgo_anode.columns = subs
Qnogo_anode = pd.read_csv(REG_DIR / 'Qnogo_anode.csv', index_col = 0); Qnogo_anode = Qnogo_anode.T; Qnogo_anode.columns = subs
Wgo_anode = pd.read_csv(REG_DIR / 'Wgo_anode.csv', index_col = 0); Wgo_anode = Wgo_anode.T; Wgo_anode.columns = subs
Wnogo_anode = pd.read_csv(REG_DIR / 'Wnogo_anode.csv', index_col = 0); Wnogo_anode = Wnogo_anode.T; Wnogo_anode.columns = subs
SV_anode = pd.read_csv(REG_DIR / 'SV_anode.csv', index_col = 0); SV_anode = SV_anode.T; SV_anode.columns = subs

Qgo_sham = pd.read_csv(REG_DIR / 'Qgo_sham.csv', index_col = 0); Qgo_sham = Qgo_sham.T; Qgo_sham.columns = subs
Qnogo_sham = pd.read_csv(REG_DIR / 'Qnogo_sham.csv', index_col = 0); Qnogo_sham = Qnogo_sham.T; Qnogo_sham.columns = subs
Wgo_sham = pd.read_csv(REG_DIR / 'Wgo_sham.csv', index_col = 0); Wgo_sham = Wgo_sham.T; Wgo_sham.columns = subs
Wnogo_sham = pd.read_csv(REG_DIR / 'Wnogo_sham.csv', index_col = 0); Wnogo_sham = Wnogo_sham.T; Wnogo_sham.columns = subs
SV_sham = pd.read_csv(REG_DIR / 'SV_sham.csv', index_col = 0); SV_sham = SV_sham.T; SV_sham.columns = subs

In [9]:
# List of tsvs
tsvs = [f for f in os.listdir(DATA_DIR) if '.tsv' in f and int(f.split('_')[0].split('-')[-1]) in subs]

In [20]:
def events(tsv, save):
    #print(tsv.split('.')[0], ' started')
    data = pd.read_csv(DATA_DIR / tsv, sep = '\t', index_col = 0)

    sub = int(tsv.split('_')[0].split('-')[-1])
    ses = int(tsv.split('_')[1].split('-')[-1])
    run = int(tsv.split('_')[3].split('-')[-1])
    
    if sub in subs:
        if sub in anode_first:
            stim = 'anode' if ses == 1 else 'sham'
        else:
            stim = 'sham'  if ses == 1 else 'anode'

        cueOnset_ls    = data['time_onset']    - data['time_scan'].iloc[0]
        fix1Onset_ls   = data['time_isi1']     - data['time_scan'].iloc[0]
        targetOnset_ls = data['time_circle']   - data['time_scan'].iloc[0]
        fix2Onset_ls   = data['time_isi2']     - data['time_scan'].iloc[0]
        fbOnset_ls     = data['time_feedback'] - data['time_scan'].iloc[0]
        waitOnset_ls   = data['time_iti']      - data['time_scan'].iloc[0]
        timeDone_ls    = data['time_done']     - data['time_scan'].iloc[0]
        cueDur_ls      = targetOnset_ls        - cueOnset_ls
        #fix1Dur_ls     = targetOnset_ls        - fix1Onset_ls
        #targetDur_ls   = fix2Onset_ls          - targetOnset_ls
        fix2Dur_ls     = fbOnset_ls            - fix2Onset_ls
        fbDur_ls       = waitOnset_ls          - fbOnset_ls
        waitDur_ls     = timeDone_ls           - waitOnset_ls

        if stim == 'anode':
            if run == 1:
                Qgo = Qgo_anode[sub][0:60].tolist();Qnogo = Qnogo_anode[sub][0:60].tolist()
                Wgo = Wgo_anode[sub][0:60].tolist();Wnogo = Wnogo_anode[sub][0:60].tolist()
                SV = SV_anode[sub][0:60].tolist()
            if run == 2:
                Qgo = Qgo_anode[sub][60:120].tolist();Qnogo = Qnogo_anode[sub][60:120].tolist()
                Wgo = Wgo_anode[sub][60:120].tolist();Wnogo = Wnogo_anode[sub][60:120].tolist()
                SV =SV_anode[sub][60:120].tolist()
            if run == 3:
                Qgo = Qgo_anode[sub][120:180].tolist();Qnogo = Qnogo_anode[sub][120:180].tolist()
                Wgo = Wgo_anode[sub][120:180].tolist();Wnogo = Wnogo_anode[sub][120:180].tolist()
                SV =SV_anode[sub][120:180].tolist()

        if stim == 'sham':
            if run == 1:
                Qgo = Qgo_sham[sub][0:60].tolist();Qnogo = Qnogo_sham[sub][0:60].tolist()
                Wgo = Wgo_sham[sub][0:60].tolist();Wnogo = Wnogo_sham[sub][0:60].tolist()
                SV =SV_sham[sub][0:60].tolist()
            if run == 2:
                Qgo = Qgo_sham[sub][60:120].tolist();Qnogo = Qnogo_sham[sub][60:120].tolist()
                Wgo = Wgo_sham[sub][60:120].tolist();Wnogo = Wnogo_sham[sub][60:120].tolist()
                SV =SV_sham[sub][60:120].tolist()
            if run == 3:
                Qgo = Qgo_sham[sub][120:180].tolist();Qnogo = Qnogo_sham[sub][120:180].tolist()
                Wgo = Wgo_sham[sub][120:180].tolist();Wnogo = Wnogo_sham[sub][120:180].tolist()
                SV =SV_sham[sub][120:180].tolist()

        tmp_ = pd.DataFrame({'cueOnset'   : cueOnset_ls,
                             'fix1Onset'  : fix1Onset_ls,
                             'targetOnset': targetOnset_ls,
                             'fix2Onset'  : fix2Onset_ls,
                             'fbOnset'    : fbOnset_ls,
                             'waitOnset'  : waitOnset_ls,
                             'cueDur'     : cueDur_ls,
                             #'fix1Dur'    : fix1Dur_ls,
                             #'targetDur'  : targetDur_ls,
                             'fbDur'      : fbDur_ls,
                             'fix2Dur'    : fix2Dur_ls,
                             'waitDur'    : waitDur_ls,
                             'Qgo'        : Qgo,
                             'Qnogo'      : Qnogo,
                             'Wgo'        : Wgo,
                             'Wnogo'      : Wnogo,
                             'SV'         : SV})

        tmp = pd.concat([tmp_, data], axis = 1)

        if save:
            fname = tsv[:4] + 'BRL' + tsv[4:]
            tmp.to_csv(OUT_DIR / 'tsvs'/ fname ,sep='\t', index = False)
            mat = fname.split('.')[0]+'.mat'
            io.savemat(str(OUT_DIR / 'mats'/ mat), {'struct':tmp.to_dict("list")},oned_as='column')

            print(fname.split('.')[0], ' saved')
        else:
            print(fname.split('.')[0], ' done')

In [21]:
for tsv in tsvs:
    events(tsv=tsv, save=True)

/home/kimhj9502/anaconda3/lib/python3.7/site-packages/scipy/io/matlab/miobase.py:414: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  arr[empties] = ' '


sub-BRL0107_ses-1_task-gng_run-2_events  saved
sub-BRL0054_ses-1_task-gng_run-2_events  saved
sub-BRL0079_ses-2_task-gng_run-2_events  saved
sub-BRL0063_ses-1_task-gng_run-2_events  saved
sub-BRL0073_ses-1_task-gng_run-1_events  saved
sub-BRL0101_ses-2_task-gng_run-2_events  saved
sub-BRL0110_ses-2_task-gng_run-1_events  saved
sub-BRL0109_ses-1_task-gng_run-2_events  saved
sub-BRL0060_ses-2_task-gng_run-1_events  saved
sub-BRL0104_ses-1_task-gng_run-2_events  saved
sub-BRL0121_ses-2_task-gng_run-1_events  saved
sub-BRL0104_ses-2_task-gng_run-1_events  saved
sub-BRL0101_ses-1_task-gng_run-2_events  saved
sub-BRL0079_ses-1_task-gng_run-3_events  saved
sub-BRL0063_ses-1_task-gng_run-1_events  saved
sub-BRL0079_ses-1_task-gng_run-2_events  saved
sub-BRL0101_ses-1_task-gng_run-1_events  saved
sub-BRL0059_ses-2_task-gng_run-1_events  saved
sub-BRL0033_ses-2_task-gng_run-3_events  saved
sub-BRL0105_ses-2_task-gng_run-1_events  saved
sub-BRL0030_ses-2_task-gng_run-3_events  saved
sub-BRL0028_s